# Pemrosesan data.csv

In [86]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [87]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/semester 5/ppw/dataset ppw/data.csv')
df.head()

,Unnamed: 0,host,time,method,url,response,bytes
0,0,***.novo.dk,805465029,GET,/ksc.html,200,7067
1,1,***.novo.dk,805465031,GET,/images/ksclogo-medium.gif,200,5866
2,2,***.novo.dk,805465051,GET,/images/MOSAIC-logosmall.gif,200,363
3,3,***.novo.dk,805465053,GET,/images/USA-logosmall.gif,200,234
4,4,***.novo.dk,805465054,GET,/images/NASA-logosmall.gif,200,786


## Filter URL yang HTML saja

In [88]:
# Filter hanya baris dengan URL berakhiran ".html"
filtered_df = df[df['url'].str.endswith('.html', na=False)].copy()

# Simpan file baru
output_file = "filter_url.csv"
filtered_df.to_csv(output_file, sep=';', index=False)

print(f"\nBerhasil menyimpan data hasil filter HTML ke {output_file}")
filtered_df.head()


Berhasil menyimpan data hasil filter HTML ke filter_url.csv


,Unnamed: 0,host,time,method,url,response,bytes
0,0,***.novo.dk,805465029,GET,/ksc.html,200,7067
6,6,***.novo.dk,805465068,GET,/shuttle/missions/missions.html,200,8678
12,12,***.novo.dk,805465381,GET,/shuttle/resources/orbiters/columbia.html,200,6922
13,13,***.novo.dk,807951768,GET,/shuttle/missions/sts-69/mission-sts-69.html,200,11264
23,23,***.novo.dk,807951938,GET,/shuttle/countdown/liftoff.html,200,4665


## Filter hanya Method GET

In [89]:
# membaca file csv yg terakhir di save
df= pd.read_csv('/content/filter_url.csv', sep=';')

get = df[df['method']=="GET"].copy()

#simpan ke csv
get.to_csv('get.csv', sep=';', index=False)
print(f"\nBerhasil menyimpan data hasil filter method GET")
get.head()


Berhasil menyimpan data hasil filter method GET


,Unnamed: 0,host,time,method,url,response,bytes
0,0,***.novo.dk,805465029,GET,/ksc.html,200,7067
1,6,***.novo.dk,805465068,GET,/shuttle/missions/missions.html,200,8678
2,12,***.novo.dk,805465381,GET,/shuttle/resources/orbiters/columbia.html,200,6922
3,13,***.novo.dk,807951768,GET,/shuttle/missions/sts-69/mission-sts-69.html,200,11264
4,23,***.novo.dk,807951938,GET,/shuttle/countdown/liftoff.html,200,4665


## Filter response == 200

In [90]:
df = pd.read_csv('/content/get.csv', sep=';')

response= df[df['response']==200].copy()
#simpan ke csv
response.to_csv('response.csv', sep=';', index=False)
print(f"\nBerhasil menyimpan data hasil filter response==200")
response.head()


Berhasil menyimpan data hasil filter response==200


,Unnamed: 0,host,time,method,url,response,bytes
0,0,***.novo.dk,805465029,GET,/ksc.html,200,7067
1,6,***.novo.dk,805465068,GET,/shuttle/missions/missions.html,200,8678
2,12,***.novo.dk,805465381,GET,/shuttle/resources/orbiters/columbia.html,200,6922
3,13,***.novo.dk,807951768,GET,/shuttle/missions/sts-69/mission-sts-69.html,200,11264
4,23,***.novo.dk,807951938,GET,/shuttle/countdown/liftoff.html,200,4665


## Merubah Request Time menjadi tanggal dan waktunya saja

In [106]:
df = pd.read_csv('/content/response.csv', sep=';')

#konversi ke datettime
df['datetime'] = pd.to_datetime(df['time'], unit='s')

#simpan ke csv
df.to_csv('datetime.csv', sep=';', index=False)
print(f"\nBerhasil menyimpan data hasil konversi time ke datetime")
df.head()


Berhasil menyimpan data hasil konversi time ke datetime


,Unnamed: 0,host,time,method,url,response,bytes,datetime
0,0,***.novo.dk,805465029,GET,/ksc.html,200,7067,1995-07-11 12:17:09
1,6,***.novo.dk,805465068,GET,/shuttle/missions/missions.html,200,8678,1995-07-11 12:17:48
2,12,***.novo.dk,805465381,GET,/shuttle/resources/orbiters/columbia.html,200,6922,1995-07-11 12:23:01
3,13,***.novo.dk,807951768,GET,/shuttle/missions/sts-69/mission-sts-69.html,200,11264,1995-08-09 07:02:48
4,23,***.novo.dk,807951938,GET,/shuttle/countdown/liftoff.html,200,4665,1995-08-09 07:05:38


## Melakukan Session untuk menemukan pola

In [92]:
import pandas as pd

# Sort dulu berdasarkan host & waktu
df = df.sort_values(by=['host', 'datetime'])

# Hitung selisih waktu antar request untuk tiap host
df['time_diff'] = df.groupby('host')['datetime'].diff()

# Tandai apakah memulai SESSION baru (jika gap > 30 menit)
df['new_session'] = (df['time_diff'] > pd.Timedelta(minutes=30)).astype(int)

# Hitung SESSION per host (cumsum)
df['SESSION'] = df.groupby('host')['new_session'].cumsum() + 1


# Step 5 - Tampilkan kolom final
df_session = df[['SESSION', 'host', 'datetime', 'url']]

# simpan ke csv
df_session.to_csv('hasil_session.csv', index=False)

df_session.head()

,SESSION,host,datetime,url
0,1,***.novo.dk,1995-07-11 12:17:09,/ksc.html
1,1,***.novo.dk,1995-07-11 12:17:48,/shuttle/missions/missions.html
2,1,***.novo.dk,1995-07-11 12:23:01,/shuttle/resources/orbiters/columbia.html
3,2,***.novo.dk,1995-08-09 07:02:48,/shuttle/missions/sts-69/mission-sts-69.html
4,2,***.novo.dk,1995-08-09 07:05:38,/shuttle/countdown/liftoff.html


In [93]:

import pandas as pd
df=pd.read_csv('/content/hasil_session.csv')


df["SESSION_ID"] = df["host"] + "_" + df["SESSION"].astype(str)


# BUAT DAFTAR HALAMAN (PAGE 1,2,3,...)
# ambil url unik
unique_pages = df["url"].unique()

# mapping: URL → PAGE_1, PAGE_2, …
page_map = {url: f"PAGE_{i+1}" for i, url in enumerate(unique_pages)}

# simpan hasil mapping sebagai kolom PAGE
df["PAGE"] = df["url"].map(page_map)



#  MEMBUAT MATRICS FREKUENSI SESI × HALAMAN
freq_matrix = df.pivot_table(
    index="SESSION_ID",
    columns="PAGE",
    values="url",
    aggfunc="count"
).fillna(0).astype(int)

# jadikan SESSION_ID sebagai kolom
freq_matrix = freq_matrix.reset_index()


# tampilkan hasil
print("===== SAMPLE MATRIX SESI × PAGE =====")
print(freq_matrix.head())

print("Jumlah kolom:", len(freq_matrix.columns))
print("Jumlah sesi:", freq_matrix.shape[0])

freq_matrix.to_csv("session_page_matrix.csv", index=False)

===== SAMPLE MATRIX SESI × PAGE =====
PAGE                 SESSION_ID  PAGE_1  PAGE_10  PAGE_100  PAGE_101  \
0                 ***.novo.dk_1       1        0         0         0   
1                 ***.novo.dk_2       0        0         0         0   
2     001.msy4.communique.net_1       0        0         0         0   
3              007.thegap.com_1       0        0         0         0   
4              007.thegap.com_2       0        1         0         0   

PAGE  PAGE_102  PAGE_103  PAGE_104  PAGE_105  PAGE_106  ...  PAGE_90  PAGE_91  \
0            0         0         0         0         0  ...        0        0   
1            0         0         0         0         0  ...        0        0   
2            0         0         0         0         0  ...        0        0   
3            0         0         0         0         0  ...        0        0   
4            0         0         0         0         0  ...        0        0   

PAGE  PAGE_92  PAGE_93  PAGE_94  PAGE_95  

# pemrosesan data webuage

In [94]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/semester 5/ppw/dataset ppw/webuage.csv')
data.head()

,Remote host,Remote logname,Remote user,Request time,Request method,Request URI,Request Protocol,Status,Size of response (incl. headers)
0,65.55.147.227,-,-,2009-10-15T02:00:24Z,GET,/index.html,HTTP/1.1,200,21878
1,65.55.86.34,-,-,2009-10-15T02:00:58Z,GET,/index.html,HTTP/1.1,200,1416
2,148.188.55.88,-,-,2009-10-15T02:01:41Z,GET,/faq.html,HTTP/1.1,200,10946
3,72.30.57.238,-,-,2009-10-15T02:01:59Z,GET,/contribute.txt,HTTP/1.0,200,39943
4,66.249.139.233,-,-,2009-10-15T02:02:09Z,GET,/faq.html,HTTP/1.1,200,17247


## filtering hanya mengambil yang html saja

In [95]:
# Filter hanya baris dengan URL berakhiran ".html"
step1 = data[data['Request URI'].str.endswith('.html', na=False)].copy()

# Simpan file baru
output_file = "step1_filter_html.csv"
step1.to_csv(output_file, sep=';', index=False)

print(f"\nBerhasil menyimpan data hasil filter HTML ke {output_file}")
step1.head()


Berhasil menyimpan data hasil filter HTML ke step1_filter_html.csv


,Remote host,Remote logname,Remote user,Request time,Request method,Request URI,Request Protocol,Status,Size of response (incl. headers)
0,65.55.147.227,-,-,2009-10-15T02:00:24Z,GET,/index.html,HTTP/1.1,200,21878
1,65.55.86.34,-,-,2009-10-15T02:00:58Z,GET,/index.html,HTTP/1.1,200,1416
2,148.188.55.88,-,-,2009-10-15T02:01:41Z,GET,/faq.html,HTTP/1.1,200,10946
4,66.249.139.233,-,-,2009-10-15T02:02:09Z,GET,/faq.html,HTTP/1.1,200,17247
5,72.30.50.248,-,-,2009-10-15T02:02:13Z,GET,/index.html,HTTP/1.0,200,7883


## Filter status == 200

In [96]:
step2 = step1[step1['Status'] == 200].copy()

output_file = "step2_status_200.csv"
step2.to_csv(output_file, sep=';', index=False)

print(f"\nBerhasil menyimpan data filter Status=200 ke {output_file}")
step2.head()


Berhasil menyimpan data filter Status=200 ke step2_status_200.csv


,Remote host,Remote logname,Remote user,Request time,Request method,Request URI,Request Protocol,Status,Size of response (incl. headers)
0,65.55.147.227,-,-,2009-10-15T02:00:24Z,GET,/index.html,HTTP/1.1,200,21878
1,65.55.86.34,-,-,2009-10-15T02:00:58Z,GET,/index.html,HTTP/1.1,200,1416
2,148.188.55.88,-,-,2009-10-15T02:01:41Z,GET,/faq.html,HTTP/1.1,200,10946
4,66.249.139.233,-,-,2009-10-15T02:02:09Z,GET,/faq.html,HTTP/1.1,200,17247
5,72.30.50.248,-,-,2009-10-15T02:02:13Z,GET,/index.html,HTTP/1.0,200,7883


## Filter request method = GET

In [97]:
step3 = step2[step2['Request method'] == "GET"].copy()

output_file = "step3_method_get.csv"
step3.to_csv(output_file, sep=';', index=False)

print(f"\nBerhasil menyimpan data filter GET ke {output_file}")
step3.head()


Berhasil menyimpan data filter GET ke step3_method_get.csv


,Remote host,Remote logname,Remote user,Request time,Request method,Request URI,Request Protocol,Status,Size of response (incl. headers)
0,65.55.147.227,-,-,2009-10-15T02:00:24Z,GET,/index.html,HTTP/1.1,200,21878
1,65.55.86.34,-,-,2009-10-15T02:00:58Z,GET,/index.html,HTTP/1.1,200,1416
2,148.188.55.88,-,-,2009-10-15T02:01:41Z,GET,/faq.html,HTTP/1.1,200,10946
4,66.249.139.233,-,-,2009-10-15T02:02:09Z,GET,/faq.html,HTTP/1.1,200,17247
5,72.30.50.248,-,-,2009-10-15T02:02:13Z,GET,/index.html,HTTP/1.0,200,7883


## Convert Request Time → datetime

In [98]:
step3["datetime"] = pd.to_datetime(step3["Request time"], format='ISO8601')

output_file = "step4_datetime.csv"
step3.to_csv(output_file, sep=';', index=False)

print(f"\nBerhasil convert datetime, disimpan ke {output_file}")
step3[['Request time', 'datetime']].head()


Berhasil convert datetime, disimpan ke step4_datetime.csv


,Request time,datetime
0,2009-10-15T02:00:24Z,2009-10-15 02:00:24+00:00
1,2009-10-15T02:00:58Z,2009-10-15 02:00:58+00:00
2,2009-10-15T02:01:41Z,2009-10-15 02:01:41+00:00
4,2009-10-15T02:02:09Z,2009-10-15 02:02:09+00:00
5,2009-10-15T02:02:13Z,2009-10-15 02:02:13+00:00


## Filter Berdasarkan Remote Host + Protocol

In [99]:
step5 = step3[step3["Request Protocol"] == "HTTP/1.1"].copy()

output_file = "step5_filter_protocol.csv"
step5.to_csv(output_file, sep=';', index=False)

print(f"\nBerhasil filter berdasarkan protocol ke {output_file}")
step5.head()


Berhasil filter berdasarkan protocol ke step5_filter_protocol.csv


,Remote host,Remote logname,Remote user,Request time,Request method,Request URI,Request Protocol,Status,Size of response (incl. headers),datetime
0,65.55.147.227,-,-,2009-10-15T02:00:24Z,GET,/index.html,HTTP/1.1,200,21878,2009-10-15 02:00:24+00:00
1,65.55.86.34,-,-,2009-10-15T02:00:58Z,GET,/index.html,HTTP/1.1,200,1416,2009-10-15 02:00:58+00:00
2,148.188.55.88,-,-,2009-10-15T02:01:41Z,GET,/faq.html,HTTP/1.1,200,10946,2009-10-15 02:01:41+00:00
4,66.249.139.233,-,-,2009-10-15T02:02:09Z,GET,/faq.html,HTTP/1.1,200,17247,2009-10-15 02:02:09+00:00
8,65.55.80.97,-,-,2009-10-15T02:02:51Z,GET,/index.html,HTTP/1.1,200,1416,2009-10-15 02:02:51+00:00


## Sort by host + datetime

In [100]:
step6 = step5.sort_values(by=["Remote host", "datetime"]).reset_index(drop=True)

output_file = "step6_sort.csv"
step6.to_csv(output_file, sep=';', index=False)

print(f"\nBerhasil sort data ke {output_file}")
step6.head()


Berhasil sort data ke step6_sort.csv


,Remote host,Remote logname,Remote user,Request time,Request method,Request URI,Request Protocol,Status,Size of response (incl. headers),datetime
0,109.192.104.209,-,-,2009-10-18T18:27:27Z,GET,/index.html,HTTP/1.1,200,19395,2009-10-18 18:27:27+00:00
1,109.192.104.86,-,-,2009-10-18T18:27:27Z,GET,/index.html,HTTP/1.1,200,99209,2009-10-18 18:27:27+00:00
2,109.192.109.233,-,-,2009-10-18T18:26:27Z,GET,/faq.html,HTTP/1.1,200,757,2009-10-18 18:26:27+00:00
3,109.192.111.243,-,-,2009-10-18T20:59:37Z,GET,/index.html,HTTP/1.1,200,10937,2009-10-18 20:59:37+00:00
4,109.192.117.151,-,-,2009-10-18T18:26:27Z,GET,/index.html,HTTP/1.1,200,285,2009-10-18 18:26:27+00:00


## Membentuk SESSION (timeout 30 menit)

In [105]:
step6["SESSION"] = 0
session_id = 0
timeout = pd.Timedelta(minutes=30)

prev_host = None
prev_time = None

for i, row in step6.iterrows():
    host = row["Remote host"]
    time = row["datetime"]

    # Host baru → session baru
    if host != prev_host:
        session_id += 1
    else:
        # Cek timeout 30 menit
        if (time - prev_time) > timeout:
            session_id += 1

    step6.at[i, "SESSION"] = session_id
    prev_host = host
    prev_time = time

output_file = "step7_session.csv"
step6.to_csv(output_file, sep=';', index=False)

print(f"\nBerhasil membentuk SESSION ke {output_file}")
step6[["Remote host", "datetime", "SESSION"]].head()


Berhasil membentuk SESSION ke step7_session.csv


,Remote host,datetime,SESSION
0,109.192.104.209,2009-10-18 18:27:27+00:00,1
1,109.192.104.86,2009-10-18 18:27:27+00:00,2
2,109.192.109.233,2009-10-18 18:26:27+00:00,3
3,109.192.111.243,2009-10-18 20:59:37+00:00,4
4,109.192.117.151,2009-10-18 18:26:27+00:00,5


In [102]:
# Bentuk SESSION_ID unik per host

step6["SESSION_ID"] = step6["Remote host"] + "_" + step6["SESSION"].astype(str)

output_file = "step8_session_id.csv"
step6.to_csv(output_file, sep=';', index=False)

print(f"\nBerhasil menambah kolom SESSION_ID ke {output_file}")
step6[["SESSION_ID", "Remote host", "datetime"]].head()



Berhasil menambah kolom SESSION_ID ke step8_session_id.csv


,SESSION_ID,Remote host,datetime
0,109.192.104.209_1,109.192.104.209,2009-10-18 18:27:27+00:00
1,109.192.104.86_2,109.192.104.86,2009-10-18 18:27:27+00:00
2,109.192.109.233_3,109.192.109.233,2009-10-18 18:26:27+00:00
3,109.192.111.243_4,109.192.111.243,2009-10-18 20:59:37+00:00
4,109.192.117.151_5,109.192.117.151,2009-10-18 18:26:27+00:00


In [103]:
# Ambil semua halaman unik
unique_pages = step6["Request URI"].unique()
page_map = {url: f"PAGE_{i+1}" for i, url in enumerate(unique_pages)}

step6["PAGE"] = step6["Request URI"].map(page_map)

output_file = "step9_page_map.csv"
step6.to_csv(output_file, sep=';', index=False)

print(f"\nBerhasil mapping halaman ke {output_file}")
step6[["Request URI", "PAGE"]].head()


Berhasil mapping halaman ke step9_page_map.csv


,Request URI,PAGE
0,/index.html,PAGE_1
1,/index.html,PAGE_1
2,/faq.html,PAGE_2
3,/index.html,PAGE_1
4,/index.html,PAGE_1


In [104]:
matrix = step6.pivot_table(
    index="SESSION_ID",
    columns="PAGE",
    values="Request URI",
    aggfunc="count"
).fillna(0).astype(int)

output_file = "step10_matrix.csv"
matrix.to_csv(output_file, sep=';')

print(f"\nBerhasil membuat matriks sesi-halaman ke {output_file}")
print(f"jumlah kolom: {len(matrix.columns)}")
print(f"jumlah sesi: {matrix.shape[0]}")
matrix.head()


Berhasil membuat matriks sesi-halaman ke step10_matrix.csv
jumlah kolom: 2
jumlah sesi: 68048


PAGE,PAGE_1,PAGE_2
SESSION_ID,,
109.192.104.209_1,1,0
109.192.104.86_2,1,0
109.192.109.233_3,0,1
109.192.111.243_4,1,0
109.192.117.151_5,1,0
